
# Deep Learning Frameworks

AY 128 (UC Berkeley; 2012-2025)


Almost all frameworks written in low-level C++/C with Python (or other scripting bindings)

### Low-level frameworks

   - Tensorflow (Google) Nov 2015. See https://www.tensorflow.org/api_docs/python/tf
   - pytorch (Python). https://pytorch.org/docs/stable/index.html
   - Theano
   - Caffe (Berkeley)
   - Torch (Lua)
   - CNTK (Microsoft)
   - Chainer
   - PaddlePaddle (Baidu) Aug 2016
   
### High-level frameworks (Python)

   - Keras (atop Tensorflow, Theano) - https://www.tensorflow.org/api_docs/python/tf/keras
   - Skorch - scikit-learn compatible neural network library that wraps PyTorch (https://github.com/skorch-dev/skorch)
   - FastAI: https://docs.fast.ai/
   - PyTorch Lightning (https://github.com/PyTorchLightning/pytorch-lightning)
   
<img src="figs/frameworks.png" width="75%">
Source: https://paperswithcode.com/trends


see also: https://github.com/mbadry1/Top-Deep-Learning

<img src="http://www.asimovinstitute.org/wp-content/uploads/2016/09/neuralnetworks.png">

Source: http://www.asimovinstitute.org/neural-network-zoo/

An example (from Josh's student work): 

<img src="https://github.com/profjsb/deepCR/raw/master/imgs/network.png">

<img src="https://raw.githubusercontent.com/profjsb/deepCR/master/imgs/postage-sm.jpg">

"deepCR: Deep Learning Based Cosmic Ray Removal for Astronomical Images"
https://github.com/profjsb/deepCR

# PyTorch

   - syntax closely resembles Python, making it easy for developers familiar with the language to transition to deep learning.
   - Dynamic Computational Graphs:
Unlike some frameworks that require you to define the entire computation graph upfront (static graphs), PyTorch builds the graph on the fly as you execute your code. This makes debugging easier and allows for more flexibility, especially in research settings where rapid prototyping is crucial.
   - PyTorch leverages the power of GPUs to accelerate computations, making it suitable for training large and complex models efficiently.                                                                           

Let's load up the California housing data as in the previous notebook.

In [ ]:
import math

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd

from sklearn import datasets
from sklearn.preprocessing import StandardScaler  

cal_data = datasets.fetch_california_housing()
X = cal_data['data']   # 8 features 
Y = cal_data['target'] # response (median house price)

half = math.floor(len(Y)/2)
train_X = X[:half]
train_Y = Y[:half]
test_X = X[half:]
test_Y = Y[half:]

# scale the data; remove mean and scale to unit variance: z = (x - u) / s
# StandardScaler transforms your data such that its distribution will have a mean value 0 and standard deviation of 1
scaler = StandardScaler()  

# Don't cheat - fit only on training data
scaler.fit(train_X)  
train_X = scaler.transform(train_X)  

# apply same transformation to test data
test_X = scaler.transform(test_X)

In [ ]:
test_X[0,:]

In [ ]:
num_input_features = train_X.shape[1]
print(f'number of input features = {num_input_features}')

In [ ]:
import datetime, os

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Print torch version
print(torch.__version__)

Let's make a simple neural network in PyTorch, as an MLP (multi-layer perceptron) with
a few layers.

In [ ]:
class NNClf(nn.Module):
    """
    A simple neural network classifier using PyTorch.

    This class defines a neural network with three hidden layers and one output layer.
    It uses ReLU (Rectified Linear Unit) activation function for the hidden layers.

    Attributes:
    -----------
    layer1 : nn.Linear
        The first linear layer which takes input features and maps them to 32 neurons.
    layer2 : nn.Linear
        The second linear layer which takes 32 neurons from the first layer and maps them to another 32 neurons.
    layer3 : nn.Linear
        The third linear layer which takes 32 neurons from the second layer and maps them to 10 neurons.
    output_layer : nn.Linear
        The output layer which takes 10 neurons from the third layer and maps them to a single output.

    Methods:
    --------
    forward(x):
        Defines the forward pass of the neural network. It takes an input tensor `x` and passes it through
        the layers with ReLU activation functions in between, and finally through the output layer.
    """
    def __init__(self, input_features):
        super(NNClf, self).__init__()
        self.layer1 = nn.Linear(input_features, 32)
        self.layer2 = nn.Linear(32, 32)
        self.layer3 = nn.Linear(32, 10)
        self.output_layer = nn.Linear(10, 1)

    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = torch.relu(self.layer2(x))
        x = torch.relu(self.layer3(x))
        x = self.output_layer(x)
        return x

In [ ]:
model = NNClf(num_input_features)
print(model)

In [ ]:
# Convert numpy arrays to PyTorch tensors
train_X_tensor = torch.tensor(train_X, dtype=torch.float32)
train_Y_tensor = torch.tensor(train_Y, dtype=torch.float32).view(-1, 1)  # reshape to a 2D tensor
test_X_tensor = torch.tensor(test_X, dtype=torch.float32)
test_Y_tensor = torch.tensor(test_Y, dtype=torch.float32).view(-1, 1)

In [ ]:
print(train_Y_tensor.size())
print(train_Y.shape)

## Datasets & DataLoaders

"Code for processing data samples can get messy and hard to maintain; we ideally want our dataset code to be decoupled from our model training code for better readability and modularity. PyTorch provides two data primitives: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset` that allow you to use pre-loaded datasets as well as your own data. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset to enable easy access to the samples."

https://pytorch.org/tutorials/beginner/basics/data_tutorial.html

In [ ]:
TensorDataset?

In [ ]:
DataLoader?

In [ ]:
# Create Dataset DataLoader
train_dataset = TensorDataset(train_X_tensor, train_Y_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

In [ ]:
# Define loss and optimizer

# The criterion is defined as Mean Squared Error (MSE) Loss, which is a common loss function 
# used for regression tasks. It measures the average squared difference between the 
# estimated values and the actual value.
criterion = nn.MSELoss()

# The optimizer is defined as Adam (Adaptive Moment Estimation) optimizer, which is an 
# algorithm for first-order gradient-based optimization of stochastic objective functions. 
# It is configured to optimize the parameters of the model with a learning rate of 0.001.
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
# Training loop
epochs = 50  # Number of epochs to train the model

for epoch in range(epochs):
    model.train()  # Set the model to training mode
    
    # Iterate over batches of data from the training loader
    for batch_X, batch_Y in train_loader:
        optimizer.zero_grad()  # Clear the gradients of all optimized tensors
        outputs = model(batch_X)  # Forward pass: compute predicted outputs by passing inputs to the model
        loss = criterion(outputs, batch_Y)  # Calculate the loss
        loss.backward()  # Backward pass: compute gradient of the loss with respect to model parameters
        optimizer.step()  # Perform a single optimization step (parameter update)
    print(f"{loss.item():0.3f}", sep=" ", end=" ", flush=True)

In [ ]:
# Evaluate the model
model.eval()  # set outselves in evaluation mode
with torch.no_grad():
    predictions = model(test_X_tensor).numpy()

In [ ]:
# Calculate MSE
mse = np.mean((predictions - test_Y_tensor.numpy()) ** 2)
print("MSE", mse)

In [ ]:
# how well did we do?
import numpy as np

plt.figure(figsize=(10,6))
plt.title("NN Regression Residuals - MSE = %.3f" % mse)
plt.scatter(test_Y_tensor.numpy(), predictions, alpha=0.4, s=3)
plt.xlabel("Test Y")
plt.ylabel("Predicted Y")
plt.plot([0.2, 5], [0.2, 5], c="r")